In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['MS']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('multiple sclerosis', 5112),
 ('metabolic syndrome', 975),
 ('mass spectrometry', 684),
 ('maternal separation', 305),
 ('methionine synthase', 92),
 ('medial septum', 90),
 ('mutans streptococci', 47),
 ('mitral stenosis', 40),
 ('microspheres', 40),
 ('mechanosensitive', 38),
 ('mass spectrometric', 34),
 ('morphine sulfate', 31),
 ('mammospheres', 24),
 ('medial septal', 21),
 ('myeloid sarcoma', 20),
 ('mechanical stress', 19),
 ('meal size', 18),
 ('mechanical stretch', 17),
 ('mass spectrometer', 17),
 ('mortality salience', 17),
 ('mainstream', 16),
 ('multiply spliced', 14),
 ('median survival', 14),
 ('malate synthase', 14),
 ('monascin', 14),
 ('mild stress', 14),
 ('methane rich saline', 14),
 ('mining waste', 14),
 ('mouse serum', 13),
 ('motion sickness', 12),
 ('mechanical stimulation', 12),
 ('muscle stimulation', 12),
 ('mental stress', 12),
 ('metoprolol succinate', 10),
 ('mycobacterium smegmatis', 10),
 ('musculoskeletal', 10),
 ('magnetic stimulation', 10),
 ('mas

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'MESH:D009103': 'Multiple Sclerosis',
 'MESH:D024821': 'Metabolic Syndrome',
 'MESH:D013058': 'Mass Spectrometry',
 'MESH:D008946': 'Mitral Valve Stenosis',
 'CHEBI:CHEBI:7003': 'morphine sulfate',
 'MESH:D023981': 'Sarcoma, Myeloid',
 'MESH:D013314': 'Stress, Mechanical',
 'EFO:0001695': 'size',
 'MESH:D013534': 'Survival',
 'CHEBI:CHEBI:82621': 'monascin',
 'EFO:0002677': 'saline',
 'MESH:D044967': 'Serum',
 'MESH:D009041': 'Motion Sickness',
 'CHEBI:CHEBI:6905': 'Metoprolol succinate',
 'MESH:D020102': 'Mycobacterium smegmatis',
 'DOID:DOID:0060056': 'hypersensitivity reaction disease',
 'MESH:D012906': 'Smoke',
 'MESH:D018895': 'Microsatellite Repeats',
 'CHEBI:CHEBI:30563': 'silicon dioxide',
 'CHEBI:CHEBI:31832': 'methyl salicylate',
 'MESH:D045805': 'Mycoplasma synoviae',
 'MESH:D053580': 'Muscle Strength',
 'MESH:D008717': 'Methionine Sulfoximine',
 'MESH:D000077323': 'Salt Stress',
 'CHEBI:CHEBI:38705': 'thiomalic acid',
 'MESH:D013232': 'Steel',
 'CHEBI:CHEBI:77966': 'food s

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-11 17:48:59] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [11]:
result

[{'m sieversii pi613981': 'ungrounded',
  'm ultiple sclerosis': 'MESH:D009103',
  'macropterous strain': 'ungrounded',
  'magnesium sulfate': 'CHEBI:CHEBI:32599',
  'magnetic stimulation': 'ungrounded',
  'magnetic susceptibility': 'ungrounded',
  'magnetic switches': 'ungrounded',
  'mainstream': 'ungrounded',
  'mainstream cigarette smoke': 'ungrounded',
  'mainstream smoke': 'ungrounded',
  'majoon suranjan': 'ungrounded',
  'malate synthase': 'ungrounded',
  'maleate salt': 'CHEBI:CHEBI:50221',
  'malignant syndrome': 'ungrounded',
  'mammospheres': 'ungrounded',
  'management system': 'ungrounded',
  'mannose sensitivity': 'ungrounded',
  'manosonication': 'ungrounded',
  'manual stimulation': 'ungrounded',
  'marbling score': 'ungrounded',
  'masking stimuli': 'ungrounded',
  'mass screening': 'MESH:D008403',
  'mass spectra': 'ungrounded',
  'mass spectrometer': 'MESH:D013058',
  'mass spectrometric': 'MESH:D013058',
  'mass spectrometry': 'MESH:D013058',
  'mass spectroscopic'

In [13]:
grounding_map, names, pos_labels = [{'m sieversii pi613981': 'ungrounded',
  'm ultiple sclerosis': 'MESH:D009103',
  'macropterous strain': 'ungrounded',
  'magnesium sulfate': 'CHEBI:CHEBI:32599',
  'magnetic stimulation': 'ungrounded',
  'magnetic susceptibility': 'ungrounded',
  'magnetic switches': 'ungrounded',
  'mainstream': 'ungrounded',
  'mainstream cigarette smoke': 'ungrounded',
  'mainstream smoke': 'ungrounded',
  'majoon suranjan': 'ungrounded',
  'malate synthase': 'ungrounded',
  'maleate salt': 'CHEBI:CHEBI:50221',
  'malignant syndrome': 'ungrounded',
  'mammospheres': 'ungrounded',
  'management system': 'ungrounded',
  'mannose sensitivity': 'ungrounded',
  'manosonication': 'ungrounded',
  'manual stimulation': 'ungrounded',
  'marbling score': 'ungrounded',
  'masking stimuli': 'ungrounded',
  'mass screening': 'MESH:D008403',
  'mass spectra': 'MESH:D013058',
  'mass spectrometer': 'MESH:D013058',
  'mass spectrometric': 'MESH:D013058',
  'mass spectrometry': 'MESH:D013058',
  'mass spectroscopic': 'MESH:D013058',
  'mass spectroscopy': 'MESH:D013058',
  'mass spectrum': 'ungrounded',
  'master station': 'ungrounded',
  'matching shoulders': 'ungrounded',
  'maternal separation': 'ungrounded',
  'maternal separation stress': 'ungrounded',
  'maxillary sinus': 'MESH:D008443',
  'meal size': 'ungrounded',
  'mean sensitivity': 'ungrounded',
  'mechanical stimulation': 'ungrounded',
  'mechanical strain': 'ungrounded',
  'mechanical stress': 'ungrounded',
  'mechanical stretch': 'ungrounded',
  'mechanosensitive': 'FPLX:Mechanosensitive_ion_channels',
  'medial septal': 'ungrounded',
  'medial septal nucleus': 'ungrounded',
  'medial septum': 'ungrounded',
  'medial septum area': 'ungrounded',
  'median survival': 'ungrounded',
  'medium salt stress': 'MESH:D000077323',
  'medium spiny': 'ungrounded',
  'meishan': 'ungrounded',
  'member state': 'ungrounded',
  'mental stress': 'ungrounded',
  'mental system': 'ungrounded',
  'mercaptosuccinic': 'CHEBI:CHEBI:38705',
  'mercaptosuccinic acid': 'CHEBI:CHEBI:38705',
  'mesoporous silica': 'ungrounded',
  'metabolic scope': 'ungrounded',
  'metabolic syndrome': 'NCIT:C84442',
  'metabolic syndrome prone': 'NCIT:C84442',
  'metam sodium': 'CHEBI:CHEBI:81737',
  'methane rich saline': 'ungrounded',
  'methanosarcinaceae': 'MESH:D017019',
  'methionine sulfoximine': 'MESH:D008717',
  'methionine synthase': 'HGNC:7468',
  'methionine synthetase': 'HGNC:7648',
  'methyl salicylate': 'CHEBI:CHEBI:31832',
  'methyl sandaracopimarate': 'ungrounded',
  'methyl supplementation': 'ungrounded',
  'methysergide': 'CHEBI:CHEBI:584020',
  'metoprolol succinate': 'CHEBI:CHEBI:6905',
  'mf and seed': 'ungrounded',
  'microsatellite': 'MESH:D018895',
  'microsclerotia': 'ungrounded',
  'microsclerotium': 'ungrounded',
  'microspheres': 'ungrounded',
  'mild steel': 'ungrounded',
  'mild stress': 'ungrounded',
  'milky spot': 'ungrounded',
  'mining seepage': 'ungrounded',
  'mining waste': 'ungrounded',
  'minoxidil sulfate': 'ungrounded',
  'mirror system': 'ungrounded',
  'mis sense': 'ungrounded',
  'missense': 'ungrounded',
  'mitral stenosis': 'ungrounded',
  'mobile sink': 'ungrounded',
  'mobile station': 'ungrounded',
  'modulating by soy flour': 'ungrounded',
  'molecular sentinel': 'ungrounded',
  'monascin': 'CHEBI:CHEBI:82621',
  'montelukast sodium': 'CHEBI:CHEBI:6993',
  'mood stabilizers': 'ungrounded',
  'morning session': 'ungrounded',
  'morning stiffness': 'ungrounded',
  'morphine sulfate': 'ungrounded',
  'mortality salience': 'ungrounded',
  'motion sickness': 'ungrounded',
  'motor sensory': 'ungrounded',
  'mouse serum': 'ungrounded',
  'ms patients': 'MESH:D009103',
  'mucosal to serosal': 'ungrounded',
  'multiple sclerosis': 'MESH:D009103',
  'multiple sclerosis patients': 'MESH:D009103',
  'multiply spliced': 'ungrounded',
  'multispectral': 'ungrounded',
  'murashige skoog': 'ungrounded',
  'muscle stimulation': 'ungrounded',
  'muscle strength': 'MESH:D053580',
  'musculoskeletal': 'ungrounded',
  'mutans streptococci': 'ungrounded',
  'mycobacterium smegmatis': 'MESH:D020102',
  'mycophenolate sodium': 'CHEBI:CHEBI:67155',
  'mycoplasma synoviae': 'MESH:D045805',
  'myeloid sarcoma': 'ungrounded',
  'strength murashige and skoog': 'ungrounded'},
 {'MESH:D009103': 'Multiple Sclerosis',
  'CHEBI:CHEBI:32599': 'magnesium sulfate',
  'CHEBI:CHEBI:50221': 'maleate salt',
  'MESH:D008403': 'Mass Screening',
  'MESH:D013058': 'Mass Spectrometry',
  'MESH:D008443': 'Maxillary Sinus',
  'FPLX:Mechanosensitive_ion_channels': 'Mechanosensitive_ion_channels',
  'MESH:D000077323': 'Salt Stress',
  'CHEBI:CHEBI:38705': 'thiomalic acid',
  'NCIT:C84442': 'Metabolic Syndrome',
  'CHEBI:CHEBI:81737': 'metam-sodium',
  'MESH:D017019': 'Methanosarcinaceae',
  'MESH:D008717': 'Methionine Sulfoximine',
  'HGNC:7468': 'MTR',
  'CHEBI:CHEBI:31832': 'methyl salicylate',
  'CHEBI:CHEBI:584020': 'methysergide',
  'CHEBI:CHEBI:6905': 'Metoprolol succinate',
  'MESH:D018895': 'Microsatellite Repeats',
  'HGNC:16228': 'TRIB3',
  'CHEBI:CHEBI:82621': 'monascin',
  'CHEBI:CHEBI:6993': 'montelukast sodium',
  'MESH:D053580': 'Muscle Strength',
  'MESH:D020102': 'Mycobacterium smegmatis',
  'CHEBI:CHEBI:67155': 'mycophenolate sodium',
  'MESH:D045805': 'Mycoplasma synoviae'},
 ['FPLX:Mechanosensitive_ion_channels', 'HGNC:7468', 'MESH:D009103']]

In [14]:
excluded_longforms = []

In [15]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [16]:
additional_entities = {}

In [17]:
unambiguous_agent_texts = {}

In [18]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [19]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-11 18:06:56] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-11 18:10:32] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9784875674922502 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [20]:
classifier.stats

{'label_distribution': {'MESH:D009103': 3455,
  'ungrounded': 705,
  'HGNC:7648': 4,
  'MESH:D013058': 600,
  'MESH:D008717': 5,
  'FPLX:Mechanosensitive_ion_channels': 32,
  'CHEBI:CHEBI:584020': 3,
  'HGNC:7468': 75,
  'CHEBI:CHEBI:31832': 5,
  'CHEBI:CHEBI:38705': 9,
  'NCIT:C84442': 674,
  'MESH:D018895': 7,
  'MESH:D020102': 8,
  'CHEBI:CHEBI:81737': 3,
  'MESH:D008443': 2,
  'CHEBI:CHEBI:82621': 13,
  'CHEBI:CHEBI:6905': 3,
  'MESH:D017019': 1,
  'MESH:D045805': 6,
  'CHEBI:CHEBI:6993': 2,
  'MESH:D008403': 2,
  'CHEBI:CHEBI:67155': 2,
  'CHEBI:CHEBI:50221': 1,
  'MESH:D053580': 7,
  'CHEBI:CHEBI:32599': 3,
  'MESH:D000077323': 1},
 'f1': {'mean': 0.978488, 'std': 0.003744},
 'precision': {'mean': 0.968514, 'std': 0.004265},
 'recall': {'mean': 0.98905, 'std': 0.003252},
 'MESH:D045805': {'f1': {'mean': 0.2, 'std': 0.4},
  'pr': {'mean': 0.2, 'std': 0.4},
  'rc': {'mean': 0.2, 'std': 0.4}},
 'MESH:D000077323': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
 

In [21]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [22]:
disamb.dump(model_name, results_path)

In [23]:
print(disamb.info())

Disambiguation model for MS

Produces the disambiguations:
	MTR*	HGNC:7468
	Mass Screening	MESH:D008403
	Mass Spectrometry	MESH:D013058
	Maxillary Sinus	MESH:D008443
	Mechanosensitive_ion_channels*	FPLX:Mechanosensitive_ion_channels
	Metabolic Syndrome	NCIT:C84442
	Methanosarcinaceae	MESH:D017019
	Methionine Sulfoximine	MESH:D008717
	Metoprolol succinate	CHEBI:CHEBI:6905
	Microsatellite Repeats	MESH:D018895
	Multiple Sclerosis*	MESH:D009103
	Muscle Strength	MESH:D053580
	Mycobacterium smegmatis	MESH:D020102
	Mycoplasma synoviae	MESH:D045805
	Salt Stress	MESH:D000077323
	TRIB3	HGNC:16228
	magnesium sulfate	CHEBI:CHEBI:32599
	maleate salt	CHEBI:CHEBI:50221
	metam-sodium	CHEBI:CHEBI:81737
	methyl salicylate	CHEBI:CHEBI:31832
	methysergide	CHEBI:CHEBI:584020
	monascin	CHEBI:CHEBI:82621
	montelukast sodium	CHEBI:CHEBI:6993
	mycophenolate sodium	CHEBI:CHEBI:67155
	thiomalic acid	CHEBI:CHEBI:38705

Class level metrics:
--------------------
Grounding                    	Count	F1     
         

In [24]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1